In [1]:
import os
import numpy as np
import sympy as sp
import torch

In [2]:
cd ../

/home/kpgelvan/SymbolicMathematics


In [3]:
from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

In [4]:
ls

CODE_OF_CONDUCT.md  main.py                          sbatch_scripts/
CONTRIBUTING.md     notebooks/                       sbatch_scripts_eval/
data/               preprocess_build_relmat_dict.py  sbatch_scripts_generate/
dumped/             preprocess_build_relmat.py       script_preprocess.sbatch
LICENSE             README.md                        split_data.py
logs/               requirements/                    src/


In [5]:
!wc -l 'data/ode1.train'

65340431 data/ode1.train


In [7]:
65_340_431

65340431

In [17]:
with open('data/ode1.train', 'r') as train_file:
    for i, line in enumerate(train_file):
        if line.split('|')[1][:3] != 'add':
            print(i, line)

104 19931|Y'	mul pow x INT- 1 add x mul a8 x

106 19669|Y'	mul x pow add x mul a8 x INT- 1

923 3601|Y'	mul x pow add mul INT+ 2 x mul a8 x INT- 1

928 3574|Y'	mul pow x INT- 1 add mul INT+ 2 x mul a8 x

1192 2876|Y'	mul pow x INT- 1 add mul INT- 1 x mul a8 x

1207 2830|Y'	mul x pow add mul INT- 1 x mul a8 x INT- 1

1733 1978|mul pow add INT+ 1 cos mul INT+ 2 x INT- 1 add mul INT- 1 sin mul INT+ 2 x add mul INT- 2 x add mul INT+ 2 Y mul Y' sin mul INT+ 2 x	add x mul a8 pow tan x INT- 1

2025 1686|Y'	mul INT+ 2 mul x pow add x mul a8 x INT- 1

2404 1349|Y'	mul div INT+ 1 INT+ 2 mul pow x INT- 1 add x mul a8 x

3415 1053|mul INT+ 3 mul pow abs x div INT- 2 INT+ 3 add mul INT+ 6 mul pow x INT+ 2 pow mul pow x INT- 1 add mul x abs Y mul INT- 1 mul abs x Y div INT+ 2 INT+ 3 mul pow INT+ 2 div INT+ 2 INT+ 3 mul pow abs x div INT+ 2 INT+ 3 add mul INT- 1 mul x Y' add mul INT- 1 mul Y sign mul pow x INT- 1 Y add mul x mul Y' sign mul pow x INT- 1 Y Y	mul x pow add a8 x INT+ 3

4114 921|Y'	mul 

178773 21|mul add mul INT+ 2 Y add mul add mul INT- 2 Y add mul x Y' mul INT- 1 mul x Y exp add INT+ 1 0 mul INT+ 2 x add mul INT- 1 mul x Y' mul INT- 1 mul x Y exp add INT- 5 mul INT- 1 x	mul a8 mul pow x INT+ 2 sinh add INT+ 5 x

178804 21|Y'	mul x pow add mul INT+ 1 5 x mul a8 x INT- 1

178868 20|Y'	mul pow x INT- 1 acos cos mul a8 x

178894 20|mul pow cosh mul pow x INT- 1 mul exp mul INT- 1 x Y INT- 2 add mul x Y add mul INT- 1 mul x Y' add mul x mul pow cosh mul pow x INT- 1 mul exp mul INT- 1 x Y INT+ 2 mul exp x tanh mul pow x INT- 1 mul exp mul INT- 1 x Y Y	mul x mul atanh mul a8 x exp x

178901 20|mul add mul x Y' add mul x Y add mul add mul x Y' add mul INT- 1 mul x Y Y exp add INT+ 1 0 mul INT+ 2 x Y exp add INT- 5 mul INT- 1 x	mul a8 mul pow x INT- 1 cosh add INT+ 5 x

179015 20|Y'	mul pow x INT- 1 mul add x mul a8 x ln INT+ 4

179631 20|mul add mul add mul INT- 1 Y Y' exp mul INT+ 2 x mul add Y' Y exp INT+ 6 exp add INT- 3 mul INT- 1 x	mul a8 cosh add INT- 3 x

179666 20|

317937 12|Y'	sin exp mul x pow add x mul a8 x INT- 1

317938 12|mul add mul INT- 1 exp mul div INT+ 1 INT+ 2 Y add mul INT+ 4 exp mul div INT+ 1 INT+ 2 x mul Y' exp mul div INT+ 1 INT+ 2 Y exp mul div INT- 1 INT+ 2 x	add x ln pow add a8 mul INT+ 2 x INT+ 2

318071 12|mul INT+ 3 mul pow abs x div INT- 2 INT+ 3 add mul INT+ 1 2 mul pow x INT+ 2 pow mul pow x INT- 1 add mul x abs add pow x INT+ 2 mul INT- 1 Y add mul pow x INT+ 2 abs x mul INT- 1 mul abs x Y div INT+ 2 INT+ 3 mul pow INT+ 2 div INT+ 2 INT+ 3 mul pow abs x div INT+ 2 INT+ 3 add pow x INT+ 2 add mul pow x INT+ 2 sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y add mul Y sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y add mul INT- 1 mul x Y' add mul INT- 1 mul x mul Y' sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y Y	mul x add x pow add a8 mul INT+ 2 x INT+ 3

318130 12|mul add mul pow Y INT+ 2 exp mul div INT+ 1 INT+ 2 exp x add mul pow Y INT+ 2 exp add mul div INT+ 1 INT+ 2 exp x mul INT+ 2 x mul INT+ 2 mul add

KeyboardInterrupt: 

In [18]:
!head 'data/ode1.train'

641556|add mul INT- 2 Y mul x Y'	mul a8 pow x INT+ 2
406394|add INT- 2 Y'	add a8 mul INT+ 2 x
265543|add mul INT- 3 Y mul x Y'	mul a8 pow x INT+ 3
238156|add INT+ 1 Y'	add a8 mul INT- 1 x
201768|add pow x INT+ 2 add mul INT- 1 mul x Y' Y	mul x add a8 x
161143|add x add mul INT- 2 Y mul x Y'	add x mul a8 pow x INT+ 2
160067|add INT- 3 Y'	add a8 mul INT+ 3 x
132622|add INT- 1 add mul INT- 2 x Y'	add a8 add x pow x INT+ 2
122995|add mul INT- 4 Y mul x Y'	mul a8 pow x INT+ 4
115829|add mul INT+ 2 Y mul x Y'	mul a8 pow x INT- 2


### поломанные строки

In [22]:
broken_lines =  {
}

In [15]:
with open('data/ode1.train', 'r') as f, open('data/ode1_cleaned.train', 'w') as f_new:
    for i, line in tqdm(enumerate(f)):
        if i in broken_lines:
            continue
        else:
            f_new.write(line)

NameError: name 'tqdm' is not defined

In [26]:
#!rm data/prim_fwd_cleaned.train

In [27]:
#!rm data/prim_fwd.train_clean

In [28]:
#!mv data/prim_fwd_cleaned.train1 data/prim_fwd.train_clean

In [30]:
!wc -l data/ode1.train_clean

45339868 data/prim_fwd.train_clean


In [31]:
!wc -l data/ode.train

45339899 data/prim_fwd.train


In [16]:
x - y

NameError: name 'x' is not defined

### паста из кода авторов

In [23]:
OPERATORS = {
        # Elementary functions
        'add': 2,
        'sub': 2,
        'mul': 2,
        'div': 2,
        'pow': 2,
        'rac': 2,
        'inv': 1,
        'pow2': 1,
        'pow3': 1,
        'pow4': 1,
        'pow5': 1,
        'sqrt': 1,
        'exp': 1,
        'ln': 1,
        'abs': 1,
        'sign': 1,
        # Trigonometric Functions
        'sin': 1,
        'cos': 1,
        'tan': 1,
        'cot': 1,
        'sec': 1,
        'csc': 1,
        # Trigonometric Inverses
        'asin': 1,
        'acos': 1,
        'atan': 1,
        'acot': 1,
        'asec': 1,
        'acsc': 1,
        # Hyperbolic Functions
        'sinh': 1,
        'cosh': 1,
        'tanh': 1,
        'coth': 1,
        'sech': 1,
        'csch': 1,
        # Hyperbolic Inverses
        'asinh': 1,
        'acosh': 1,
        'atanh': 1,
        'acoth': 1,
        'asech': 1,
        'acsch': 1,
        # Derivative
        'derivative': 2,
        # custom functions
        'f': 1,
        'g': 2,
        'h': 3,
    }

symbols = ['I', 'INT+', 'INT-', 'INT', 'FLOAT', '-', '.', '10^']

In [24]:
constants = ['pi', 'E']
variables = ['x', 'y', 'z', 't', 'Y', "Y'", "Y''"]
functions = ['f', 'g', 'h']
elements = [str(i) for i in range(-10, 10)]
coefficients = [f'a{i}' for i in range(10)]

In [25]:
no_child_symbols = constants + variables + functions + elements + coefficients

### Черновик для создания adj matrices

In [195]:
from tqdm import tqdm
import queue

def get_ancestors(exp_list, exp_len):
    q = queue.LifoQueue()
    q.put(-1)                            # so last element gets this parent but doesn't save it

    ancestors = {0: []}
    node2parent = {}
    levels = {0: -1}

    parent = 0
    for i in range(exp_len):
        op_now = exp_list[i]

        node2parent[i] = parent
        try:
            levels[i] = levels[parent] + 1
        except:
            print('you are in except')
            return {}, {}


        if op_now in OPERATORS or op_now in symbols:   # <=> node has children
            if op_now in OPERATORS and OPERATORS[op_now] == 2:    # <=> node has 2 children
                q.put(i)
            parent = i
        elif op_now in no_child_symbols:
            if op_now.isdigit() and i + 1 < exp_len and exp_list[i + 1].isdigit():   # e.g. 18
                parent = i
            else:
                parent = q.get()
        else:
            print(op_now)
            #raise(NotFound)
            return False
        ancestors[i] = [i] + ancestors[node2parent[i]]

    return ancestors, levels

In [196]:
def get_path(i, j):
    if i == j:
        return "<self>"
    anc_i = set(ancestors[i])
      
    for node in ancestors[j][-(levels[i] + 1) :]:
        if node in anc_i:
            up_n = levels[i] - levels[node]
            down_n = levels[j] - levels[node]
            return str(up_n + 0.001 * down_n)

In [197]:
def get_ud_masks(ancestors, levels, exp_len):
    path_rels = []
    for i in range(exp_len):
        path_rels.append(" ".join([get_path(i, j) for j in range(exp_len)]))
    
    return path_rels

###

In [198]:
F_prefix  = ['mul', 'x', 'tan', 'mul', 'pow', 'x', 'INT-', '1', 'exp', 'x']

In [199]:
ancestors, levels = get_ancestors(F_prefix, len(F_prefix))
rel_matrix = get_ud_masks(ancestors, levels, len(F_prefix))
rel_matrix

['<self> 0.001 0.001 0.002 0.003 0.004 0.004 0.005 0.003 0.004',
 '1.0 <self> 1.001 1.002 1.003 1.004 1.004 1.005 1.003 1.004',
 '1.0 1.001 <self> 0.001 0.002 0.003 0.003 0.004 0.002 0.003',
 '2.0 2.001 1.0 <self> 0.001 0.002 0.002 0.003 0.001 0.002',
 '3.0 3.001 2.0 1.0 <self> 0.001 0.001 0.002 1.001 1.002',
 '4.0 4.001 3.0 2.0 1.0 <self> 1.001 1.002 2.001 2.002',
 '4.0 4.001 3.0 2.0 1.0 1.001 <self> 0.001 2.001 2.002',
 '5.0 5.001 4.0 3.0 2.0 2.001 1.0 <self> 3.001 3.002',
 '3.0 3.001 2.0 1.0 1.001 1.002 1.002 1.003 <self> 0.001',
 '4.0 4.001 3.0 2.0 2.001 2.002 2.002 2.003 1.0 <self>']

### подсчет сколько брать tree_rel_vocab_size

In [28]:
#import jsonlines
import json
from tqdm import tqdm

In [29]:
from collections import Counter

def _insert(iterable):
    words = []
    for w in iterable:
        words.append(w)
    word_count.update(words)

word_count = Counter()
with open("data/", 'r') as f:
    for line in tqdm(f):
        matrix = json.loads(line)
        new_matrix = [line.split() for line in matrix]
        for tokens in new_matrix:
            for elem in tokens:
                _insert(elem.split("_"))

45339871it [11:49:50, 1064.56it/s]


In [30]:
len(word_count)

26317

In [37]:
sorted(list(word_count.values()))[::-1][:1000][-10:]

[1120, 1120, 1112, 1112, 1111, 1111, 1106, 1106, 1104, 1104]

In [38]:
sorted(list(word_count.values()))[::-1][:2000][-10:]

[219, 219, 218, 218, 218, 218, 217, 217, 217, 217]

In [39]:
sorted(list(word_count.values()))[::-1][:5000][-10:]

[29, 29, 29, 29, 29, 28, 28, 28, 28, 28]

In [40]:
sorted(list(word_count.values()))[::-1][:10000][-10:]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [45]:
sorted(list(word_count.values()))[::-1][:11000][-10:]

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

In [46]:
sorted(list(word_count.values()))[::-1][:12000][-10:]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

### Generating tree_rel_vocab

In [38]:
special_tokens = "unk"
num_spec_tokens = len(special_tokens.split("_"))
# -2 to reserve spots for PAD and UNK token
dict_size = 500
dict_size = dict_size - num_spec_tokens if dict_size and dict_size > num_spec_tokens else dict_size
most_common = word_count.most_common(dict_size)

In [40]:
values = np.array(list(word_count.values()))
keys = np.array(list(word_count.keys()))
idxs = np.argsort(values)[::-1]
for i, (key, value) in enumerate(zip(keys[idxs], values[idxs])):
    print(value, key, i)

215235 <self> 0
205250 0.001 1
205250 1.0 2
195265 0.002 3
195265 2.0 4
185280 0.003 5
185280 3.0 6
180218 2.002 7
165652 4.0 8
165652 0.004 9
157936 2.003 10
157936 3.002 11
149647 2.001 12
149647 1.002 13
135328 0.005 14
135328 5.0 15
126700 1.003 16
126700 3.001 17
126184 3.003 18
125676 1.001 19
110286 2.004 20
110286 4.002 21
98612 0.006 22
98612 6.0 23
93767 4.001 24
93767 1.004 25
75788 4.003 26
75788 3.004 27
68211 2.005 28
68211 5.002 29
63435 0.007 30
63435 7.0 31
61036 1.005 32
61036 5.001 33
42788 3.005 34
42788 5.003 35
37129 6.002 36
37129 2.006 37
35786 8.0 38
35786 0.008 39
35132 4.004 40
34561 1.006 41
34561 6.001 42
21927 3.006 43
21927 6.003 44
18021 2.007 45
18021 7.002 46
17859 0.009000000000000001 47
17859 9.0 48
17232 1.007 49
17232 7.001 50
16357 5.004 51
16357 4.005 52
10241 3.007 53
10241 7.003 54
8004 0.01 55
8004 10.0 56
7820 2.008 57
7820 8.002 58
7687 1.008 59
7687 8.001 60
7364 6.004 61
7364 4.006 62
6746 5.005 63
4343 3.008 64
4343 8.003 65
3315 0.011 66

In [42]:
values = np.array(list(word_count.values()))
keys = np.array(list(word_count.keys()))
idxs = np.argsort(values)[::-1]
vocab = []
for i, (key, value) in enumerate(zip(keys[idxs], values[idxs])):
    print(value, key, i)
    vocab.append(key)
with open("rel_vocab.txt", "w") as fout:
    fout.write("\n".join(vocab))

215235 <self> 0
205250 0.001 1
205250 1.0 2
195265 0.002 3
195265 2.0 4
185280 0.003 5
185280 3.0 6
180218 2.002 7
165652 4.0 8
165652 0.004 9
157936 2.003 10
157936 3.002 11
149647 2.001 12
149647 1.002 13
135328 0.005 14
135328 5.0 15
126700 1.003 16
126700 3.001 17
126184 3.003 18
125676 1.001 19
110286 2.004 20
110286 4.002 21
98612 0.006 22
98612 6.0 23
93767 4.001 24
93767 1.004 25
75788 4.003 26
75788 3.004 27
68211 2.005 28
68211 5.002 29
63435 0.007 30
63435 7.0 31
61036 1.005 32
61036 5.001 33
42788 3.005 34
42788 5.003 35
37129 6.002 36
37129 2.006 37
35786 8.0 38
35786 0.008 39
35132 4.004 40
34561 1.006 41
34561 6.001 42
21927 3.006 43
21927 6.003 44
18021 2.007 45
18021 7.002 46
17859 0.009000000000000001 47
17859 9.0 48
17232 1.007 49
17232 7.001 50
16357 5.004 51
16357 4.005 52
10241 3.007 53
10241 7.003 54
8004 0.01 55
8004 10.0 56
7820 2.008 57
7820 8.002 58
7687 1.008 59
7687 8.001 60
7364 6.004 61
7364 4.006 62
6746 5.005 63
4343 3.008 64
4343 8.003 65
3315 0.011 66

## Generating adj matrices

In [229]:
### НЕПРАВИЛЬНО, НО ДЛЯ TREE REL ATT ТАК

symbols = ['I', 'INT+', 'INT-', 'INT', 'FLOAT', '-', '.', '10^', 'Y', "Y'", "Y''"]

constants = ['pi', 'E']
variables = ['x', 'y', 'z', 't']
functions = ['f', 'g', 'h']
elements = [str(i) for i in range(-10, 10)]
coefficients = [f'a{i}' for i in range(10)]

no_child_symbols = constants + variables + functions + elements + coefficients

In [230]:
import json
#import jsonlines

for set_name in ['no_dups_valtest']:
    with open('data/prim_fwd.' + set_name, 'r') as expressions:
        with open('data/rel_matrix_'+set_name+'.json', 'w') as rel_matrix_json:
            for i, line in tqdm(enumerate(expressions)):
                #print(line)
                qa = line.split('|')[1].split('\t')
                if len(qa) == 2:
                    q, a = qa
                else:
                    print(i,'is broken')
                    continue
                #print(q, ';', a)
                
                q = q.split()
                #a = a.split()
                
                ance_lev = get_ancestors(q, len(q))
                if len(ance_lev) == 2:
                    ancestors, levels = ance_lev
                else:
                    print(i, 'is broken')
                    continue
                rel_matrix_q = get_ud_masks(ancestors, levels, len(q))              

                #ancestors, levels = get_ancestors(a, len(a))
                #rel_matrix_a = get_ud_masks(ancestors, levels, len(a))
                
                rel_matrix_json.write(json.dumps(rel_matrix_q) + '\n')

7737it [00:07, 1084.41it/s]


In [130]:
### было
!head -n 5 'data/rel_matrix_test.json'

["<self> 0.001 0.002 0.003 0.003 0.004 0.004 0.005", "1.0 <self> 0.001 0.002 0.002 0.003 0.003 0.004", "2.0 1.0 <self> 0.001 0.001 0.002 0.002 0.003", "3.0 2.0 1.0 <self> 1.001 1.002 1.002 1.003", "3.0 2.0 1.0 1.001 <self> 0.001 0.001 0.002", "4.0 3.0 2.0 2.001 1.0 <self> 1.001 1.002", "4.0 3.0 2.0 2.001 1.0 1.001 <self> 0.001", "5.0 4.0 3.0 3.001 2.0 2.001 1.0 <self>"]
["<self> 0.001 0.002 0.003 0.003 0.004", "1.0 <self> 0.001 0.002 0.002 0.003", "2.0 1.0 <self> 0.001 0.001 0.002", "3.0 2.0 1.0 <self> 1.001 1.002", "3.0 2.0 1.0 1.001 <self> 0.001", "4.0 3.0 2.0 2.001 1.0 <self>"]
["<self> 0.001 0.002 0.003 0.003 0.004 0.004 0.005", "1.0 <self> 0.001 0.002 0.002 0.003 0.003 0.004", "2.0 1.0 <self> 0.001 0.001 0.002 0.002 0.003", "3.0 2.0 1.0 <self> 1.001 1.002 1.002 1.003", "3.0 2.0 1.0 1.001 <self> 0.001 0.001 0.002", "4.0 3.0 2.0 2.001 1.0 <self> 1.001 1.002", "4.0 3.0 2.0 2.001 1.0 1.001 <self> 0.001", "5.0 4.0 3.0 3.001 2.0 2.001 1.0 <self>"]
["<self> 0.001 0.002 0.003 0.004 0.005 

In [125]:
### стало
!head -n 5 'data/rel_matrix_test.json'

["<self> 0.001 0.001 0.002 0.002 0.003 0.003 0.004", "1.0 <self> 1.001 1.002 1.002 1.003 1.003 1.004", "1.0 1.001 <self> 0.001 0.001 0.002 0.002 0.003", "2.0 2.001 1.0 <self> 1.001 1.002 1.002 1.003", "2.0 2.001 1.0 1.001 <self> 0.001 0.001 0.002", "3.0 3.001 2.0 2.001 1.0 <self> 1.001 1.002", "3.0 3.001 2.0 2.001 1.0 1.001 <self> 0.001", "4.0 4.001 3.0 3.001 2.0 2.001 1.0 <self>"]
["<self> 0.001 0.001 0.002 0.002 0.003", "1.0 <self> 1.001 1.002 1.002 1.003", "1.0 1.001 <self> 0.001 0.001 0.002", "2.0 2.001 1.0 <self> 1.001 1.002", "2.0 2.001 1.0 1.001 <self> 0.001", "3.0 3.001 2.0 2.001 1.0 <self>"]
["<self> 0.001 0.001 0.002 0.002 0.003 0.003 0.004", "1.0 <self> 1.001 1.002 1.002 1.003 1.003 1.004", "1.0 1.001 <self> 0.001 0.001 0.002 0.002 0.003", "2.0 2.001 1.0 <self> 1.001 1.002 1.002 1.003", "2.0 2.001 1.0 1.001 <self> 0.001 0.001 0.002", "3.0 3.001 2.0 2.001 1.0 <self> 1.001 1.002", "3.0 3.001 2.0 2.001 1.0 1.001 <self> 0.001", "4.0 4.001 3.0 3.001 2.0 2.001 1.0 <self>"]
["<self

### Черновик для создания root paths

In [34]:
F_prefix = ['add', 'mul', 'INT-', '3', 'Y', 'mul', 'x', "Y'"]  # mul a8 pow x INT+ 3

In [35]:
from tqdm import tqdm
import queue
from collections import OrderedDict, deque

def get_root_paths(exp_list, exp_len):
    q = deque()
    q.append(-1)                            # so last element gets this parent but doesn't save it

    root_paths = OrderedDict([(i, '') for i in range(-1, len(exp_list) + 1)])   # init with empty lines to be able to add
    prev_is_digit = False
    is_right, is_down = False, False

    parent = 0
    for i in range(exp_len):
        op_now = exp_list[i]
        
        if prev_is_digit:
            if op_now.isdigit():   # e.g. 18
                parent = i - 1
            else:
                parent = q.pop()
                is_right = True
            prev_is_digit = False
            
        if i != 0:
            if parent == -1:
                print('ohoh')
                return -1
            root_paths[i] += root_paths[parent]
            if is_right:
                last_step = '2'     # right
            elif is_down:
                last_step = '0'     # down
            else:
                last_step = '1'     # left

            root_paths[i] += last_step
            is_right, is_down = False, False
        

        if op_now in OPERATORS or op_now in symbols:   # <=> node has children
            if op_now in OPERATORS and OPERATORS[op_now] == 2:    # <=> node has 2 children
                q.append(i)
            else:
                is_down = True
            parent = i
        elif op_now in no_child_symbols:
            if op_now.isdigit() and i + 1 < exp_len:
                prev_is_digit = True
            else:
                parent = q.pop()
                is_right = True

                
    return root_paths

In [36]:
root_paths = get_root_paths(F_prefix, len(F_prefix))

root_paths

OrderedDict([(-1, ''),
             (0, ''),
             (1, '1'),
             (2, '11'),
             (3, '110'),
             (4, '12'),
             (5, '2'),
             (6, '21'),
             (7, '22'),
             (8, '')])

In [37]:
root_paths = get_root_paths(F_prefix, len(F_prefix))



for i, key in enumerate(root_paths):
    if root_paths[key] == '':
        print('root')
    else:
        print('/'.join(root_paths[key]))

root
root
1
1/1
1/1/0
1/2
2
2/1
2/2
root


### Реал пишу в файлы:

In [41]:
for set_name in ['no_dups_valtest', 'valid', 'test', 'train']:
    with open('data/ode1.' + set_name, 'r') as expressions:
        with open('data/ODE_root_paths_QA'+set_name+'.json', 'w') as root_paths:
            first = True
            for i, line in tqdm(enumerate(expressions)):
                # print(line)
                qa = line.split('|')[1].split('\t')
                if qa.__len__() != 2:
                    print('line is broken', i)
                    continue
                q, a = qa
                q = q.split()
                a = a.split()
                
                paths_q = get_root_paths(q, len(q))
                paths_a = get_root_paths(a, len(a))
                if paths_q == -1 or paths_a == -1:
                    print('line is broken', i)
                    continue
                paths_q = [paths_q[path] if paths_q[path] != '' else 'root' for i, path in enumerate(paths_q)]
                paths_a = [paths_a[path] if paths_a[path] != '' else 'root' for i, path in enumerate(paths_a)]
                root_paths.write(("\n" if not first else "") + " ".join(paths_q) + "\t" + " ".join(paths_a))
                first = False
                # if i >= 5: break
            root_paths.write("\n")

13797it [00:04, 2958.29it/s]
7994it [00:02, 3214.18it/s]
7968it [00:02, 3217.55it/s]
65340431it [2:52:35, 6309.87it/s]


## Checking same to const in train/val/test

In [159]:
pip install psutil --user

Note: you may need to restart the kernel to use updated packages.


In [164]:
import psutil
import sys
from tqdm import tqdm

print(psutil.virtual_memory().percent)

61.5


In [165]:
constants = set(elements)
print(constants)

{'-7', '-6', '7', '6', '1', '-9', '-2', '-10', '-4', '2', '-3', '-5', '-1', '-8', '8', '0', '3', '9', '5', '4'}


### QA

In [166]:
def remove_const_in_line(line, cleaned_str):
    line = line.split()
    for j in range(len(line) - 1):
        word = line[j]
        if word in constants:
            if line[j+1] in constants:
                continue
            else:
                cleaned_str += ' C'
        else:
            cleaned_str += ' '
            cleaned_str += word
    if line[-1] in constants:
        cleaned_str += ' C'
    
    return cleaned_str

In [167]:
def how_many_in_set(path, sets, ALL_TRAIN_SET, same_lines_file):
    
    same_lines_num = 0

    for set_name in sets:
        with open(path + set_name, 'r') as expressions:
            for i, line in tqdm(enumerate(expressions)):
                qa = line.split('|')[1].split('\t')
                if len(qa) == 2:
                    q, a = qa
                else:
                    print(i,'is broken')
                    continue

                cleaned_str = ''
                cleaned_str = remove_const_in_line(q, cleaned_str)

                cleaned_str += '\t'
                cleaned_str += remove_const_in_line(a, cleaned_str)

                if cleaned_str in ALL_TRAIN_SET:
                    #print(i, q, a)
                    #print(cleaned_str)
                    #print()
                    
                    if same_lines_file is not None:
                        print(set_name, i, file=same_lines_file)
                    same_lines_num += 1


    print(same_lines_num)

In [19]:
ALL_TRAIN_SET = set()

with open('data/prim_fwd.' + 'train_clean', 'r') as expressions:
    for i, line in tqdm(enumerate(expressions)):
        qa = line.split('|')[1].split('\t')
        if len(qa) == 2:
            q, a = qa
        else:
            print(i,'is broken')
            continue

        cleaned_str = ''
        cleaned_str = remove_const_in_line(q, cleaned_str)

        cleaned_str += '\t'
        cleaned_str += remove_const_in_line(a, cleaned_str)

        ALL_TRAIN_SET.add(cleaned_str)
        
print(sys.getsizeof(ALL_TRAIN_SET))

45339871it [14:12, 53198.62it/s]

1073742056


In [20]:
len(ALL_TRAIN_SET)

30764137

In [1]:
30_764_137

30764137

In [21]:
with open('data/same_lines_qa.txt', 'w') as same_lines_file:
    how_many_in_set(path='data/prim_fwd.', 
                    sets=['valid', 'test'], 
                    ALL_TRAIN_SET=ALL_TRAIN_SET,
                    same_lines_file=same_lines_file)

9985it [00:00, 55866.91it/s]
9986it [00:00, 56270.19it/s]

9836


#### Добавляем в ALL_TRAIN_SET значения из val и test для генерации своих совсем новых:

In [22]:
for set_now in ['valid', 'test']:
    with open('data/prim_fwd.' + set_now, 'r') as expressions:
        for i, line in tqdm(enumerate(expressions)):
            qa = line.split('|')[1].split('\t')
            if len(qa) == 2:
                q, a = qa
            else:
                print(i,'is broken')
                continue

            cleaned_str = ''
            cleaned_str = remove_const_in_line(q, cleaned_str)

            cleaned_str += '\t'
            cleaned_str += remove_const_in_line(a, cleaned_str)

            ALL_TRAIN_SET.add(cleaned_str)

    print(sys.getsizeof(ALL_TRAIN_SET))

9985it [00:00, 56379.08it/s]
0it [00:00, ?it/s]

1073742056


9986it [00:00, 57009.82it/s]

1073742056


In [23]:
len(ALL_TRAIN_SET)

30774272

In [24]:
30774272 - 30764137

10135

добавилось сколько и ждали

### Создаем no_dups_valtest: 

In [226]:
set2same_lines_num = {'valid':[], 'test':[]}

with open('data/' + 'same_lines.txt', 'r') as same_lines_file:
    for line in tqdm(same_lines_file):
        set_name, str_num = line.split()
        set2same_lines_num[set_name].append(int(str_num))

12234it [00:00, 1050680.11it/s]


In [227]:
with open('data/prim_fwd.' + 'no_dups_valtest', 'w') as no_dups_file:
    
    for set_name in ['valid', 'test']:
        with open('data/prim_fwd.' + set_name, 'r') as expressions:
                for i, line in tqdm(enumerate(expressions)):
                    if i not in set2same_lines_num[set_name]:
                        print(line, file=no_dups_file, end='')

9985it [00:00, 22187.43it/s]
9986it [00:00, 22547.07it/s]


In [228]:
!wc -l 'data/prim_fwd.no_dups_valtest'

7737 data/prim_fwd.no_dups_valtest


### Заходим в директорию со сгенерированными данными и создаем свой вал-тест

In [26]:
cd sbatch_scripts_generate/dumped/prim_FWD_no_dups/

/home/kpgelvan/SymbolicMathematics/sbatch_scripts_generate/dumped/prim_FWD_no_dups


In [27]:
cd 239019/

/home/kpgelvan/SymbolicMathematics/sbatch_scripts_generate/dumped/prim_FWD_no_dups/239019


In [28]:
ls

data.infix  data.prefix  params.pkl  train.log


In [29]:
def how_many_in_set_no_split(path, sets, ALL_TRAIN_SET, same_lines_file):
    
    same_lines_num = 0

    for set_name in sets:
        with open(path + set_name, 'r') as expressions:
            for i, line in tqdm(enumerate(expressions)):
                qa = line.split('\t')                       ### no split by |
                if len(qa) == 2:
                    q, a = qa
                else:
                    print(i, 'is broken')
                    continue

                cleaned_str = ''
                cleaned_str = remove_const_in_line(q, cleaned_str)

                cleaned_str += '\t'
                cleaned_str += remove_const_in_line(a, cleaned_str)

                if cleaned_str in ALL_TRAIN_SET:
                    #print(i, q, a)
                    #print(cleaned_str)
                    #print()
                    
                    if same_lines_file is not None:
                        print(set_name, i, file=same_lines_file)
                    same_lines_num += 1


    print(same_lines_num)

In [30]:
with open('same_lines_here.txt', 'w') as same_lines_file:
    how_many_in_set_no_split(path='',
                    sets=['data.prefix'], 
                    ALL_TRAIN_SET=ALL_TRAIN_SET,
                    same_lines_file=same_lines_file)

306368it [00:03, 82713.25it/s]

238960


C помощью скрипта авторов создаем файл data.prefix.counts вида 

```кол-во_повторений | q a```

Записываем повторяющиеся строки в новый файл ..._clear.txt

In [31]:
%%bash

cat data.prefix \
| awk 'BEGIN{PROCINFO["sorted_in"]="@val_num_desc"}{c[$0]++}END{for (i in c) printf("%i|%s\n",c[i],i)}' \
> data.prefix.counts

In [33]:
with open('same_lines_here_clear.txt', 'w') as same_lines_file:
    how_many_in_set(path='',
                    sets=['data.prefix.counts'], 
                    ALL_TRAIN_SET=ALL_TRAIN_SET,
                    same_lines_file=same_lines_file)

195880it [00:02, 70768.54it/s]

128485


In [42]:
# Получаем номера повторных строк 
same_lines_nums = set()
with open('same_lines_here_clear.txt', 'r') as same_lines_file:
    for line in same_lines_file:
        same_lines_nums.add(int(line.split(' ')[1]))
print(len(same_lines_nums))

# пишем в новый файл без повторов
with open('data.prefix.counts', 'r') as samples:
    with open('data.prefix.counts_clear', 'w') as samples_clear:
        for i, line in enumerate(samples):
            if i not in same_lines_nums:
                print(line, file=samples_clear, end='')

128485


In [43]:
!wc -l data.prefix.counts_clear

67395 data.prefix.counts_clear


##### Итого получили 67_000 новых примеров, по 10_000 из которых отправим в тест и вал:

In [ ]:
#!head -n 10000 data.prefix.counts_clear > ../../../../data/prim_fwd.mytest10k
#!tail -n 10000 data.prefix.counts_clear > ../../../../data/prim_fwd.myval10k

### Вспомогательный эксперимент

Сделан для подбора параметров генерации вал и тест сетов авторами: 

In [73]:
#'data/prim_fwd.train_clean'    -> OPERATORS[el] == 2  => MAX=50; IN OP => MAX=67,mean=10
#'sbatch_scripts_generate/dumped/prim_FWD_no_dups/239019/data.prefix.counts_clear' -> OPERATORS[el] == 2  => MAX=26, IN OP => MAX=30, mean=10

In [74]:
all_num_pos = []

with open('data/prim_fwd.train_clean', 'r') as expressions:                         # 'data/prim_fwd.train_clean'
    for i, line in tqdm(enumerate(expressions)):
        qa = line.split('|')[1].split('\t')
        if len(qa) == 2:
            q, a = qa
        else:
            print(i,'is broken')
            continue
            
        q = q.split()
        num_ops = 0
        for el in q:
            if el in OPERATORS:
                num_ops += 1
        all_num_pos.append(num_ops)

max(all_num_pos)









0it [00:00, ?it/s]







29249it [00:00, 292486.79it/s]







57574it [00:00, 289651.50it/s]







87786it [00:00, 293279.66it/s]







115215it [00:00, 287311.52it/s]







145547it [00:00, 291932.26it/s]







170763it [00:00, 274005.67it/s]







199797it [00:00, 278707.46it/s]







229430it [00:00, 283769.83it/s]







258937it [00:00, 287066.23it/s]







286586it [00:01, 277124.10it/s]







315079it [00:01, 279418.51it/s]







344091it [00:01, 282543.64it/s]







373219it [00:01, 285107.11it/s]







401515it [00:01, 276814.62it/s]







430422it [00:01, 280379.24it/s]







459273it [00:01, 282767.64it/s]







487515it [00:01, 277902.25it/s]







515854it [00:01, 279523.64it/s]







544365it [00:01, 281174.32it/s]







572921it [00:02, 282473.70it/s]







601171it [00:02, 275169.43it/s]







629723it [00:02, 278194.43it/s]







658159it [00:02, 280013.36it/s]







686190it [00:02, 273860.95it/s]







714079it [00:02, 275348.07it/s]



6206966it [00:26, 195172.69it/s]







6226810it [00:26, 196140.34it/s]







6246435it [00:26, 194232.82it/s]







6266806it [00:26, 196979.54it/s]







6287138it [00:26, 198838.43it/s]







6307039it [00:26, 196306.41it/s]







6327325it [00:26, 198226.68it/s]







6347691it [00:26, 199823.24it/s]







6367689it [00:27, 195927.98it/s]







6387310it [00:27, 191875.77it/s]







6407281it [00:27, 194159.44it/s]







6426731it [00:27, 193270.07it/s]







6447082it [00:27, 196231.57it/s]







6467616it [00:27, 198875.55it/s]







6487532it [00:27, 195851.72it/s]







6507945it [00:27, 198263.20it/s]







6527800it [00:27, 196603.75it/s]







6547990it [00:27, 198161.88it/s]







6568169it [00:28, 199236.32it/s]







6588108it [00:28, 196757.12it/s]







6608113it [00:28, 197733.32it/s]







6628429it [00:28, 199329.86it/s]







6648375it [00:28, 190421.03it/s]







6668431it [00:28, 193352.68it/s]







6688515it [00:28, 195537.20it/s]









11295054it [00:52, 195553.10it/s]







11315472it [00:52, 198061.36it/s]







11335295it [00:52, 195706.18it/s]







11355555it [00:52, 197722.82it/s]







11375825it [00:52, 199188.91it/s]







11395759it [00:52, 196083.55it/s]







11415571it [00:52, 196687.90it/s]







11435257it [00:52, 196736.46it/s]







11454942it [00:52, 193827.53it/s]







11475127it [00:53, 196164.65it/s]







11495305it [00:53, 197812.61it/s]







11515103it [00:53, 195101.59it/s]







11535045it [00:53, 196377.12it/s]







11555167it [00:53, 197804.83it/s]







11574962it [00:53, 195086.17it/s]







11595202it [00:53, 197222.76it/s]







11615430it [00:53, 198711.98it/s]







11635317it [00:53, 195452.09it/s]







11655022it [00:53, 195925.17it/s]







11674738it [00:54, 196292.37it/s]







11694379it [00:54, 188970.52it/s]







11714581it [00:54, 192704.77it/s]







11734744it [00:54, 195295.47it/s]







11754328it [00:54, 193479.34it/s]







11774180it [00:5

16293368it [01:17, 193909.04it/s]







16312766it [01:17, 192673.45it/s]







16332387it [01:18, 193720.05it/s]







16352775it [01:18, 196659.61it/s]







16372457it [01:18, 195600.21it/s]







16392268it [01:18, 196344.69it/s]







16412460it [01:18, 197983.21it/s]







16432268it [01:18, 187528.33it/s]







16452480it [01:18, 191678.21it/s]







16472427it [01:18, 193950.05it/s]







16491904it [01:18, 192643.35it/s]







16512115it [01:18, 195388.28it/s]







16532221it [01:19, 197052.65it/s]







16551966it [01:19, 194890.39it/s]







16572122it [01:19, 196842.87it/s]







16592511it [01:19, 198903.99it/s]







16612426it [01:19, 196310.58it/s]







16632663it [01:19, 198089.87it/s]







16652901it [01:19, 199356.34it/s]







16672854it [01:19, 195428.52it/s]







16692557it [01:19, 195906.14it/s]







16712224it [01:19, 196134.45it/s]







16731852it [01:20, 185445.01it/s]







16751503it [01:20, 188629.53it/s]







16771260it [01:2

21280228it [01:43, 192835.50it/s]







21299535it [01:43, 192122.96it/s]







21319718it [01:43, 194934.30it/s]







21340033it [01:43, 197327.06it/s]







21359788it [01:43, 195603.30it/s]







21379829it [01:43, 197016.69it/s]







21400125it [01:44, 198762.03it/s]







21420016it [01:44, 195577.84it/s]







21439596it [01:44, 195327.60it/s]







21459339it [01:44, 195953.24it/s]







21478946it [01:44, 194570.37it/s]







21499183it [01:44, 196843.71it/s]







21519010it [01:44, 197267.39it/s]







21538747it [01:44, 195369.54it/s]







21558893it [01:44, 197157.16it/s]







21579106it [01:45, 198622.49it/s]







21598979it [01:45, 195091.71it/s]







21618951it [01:45, 196455.08it/s]







21639074it [01:45, 197860.73it/s]







21658875it [01:45, 194911.72it/s]







21678786it [01:45, 196152.28it/s]







21698757it [01:45, 197205.99it/s]







21718491it [01:45, 192870.95it/s]







21738287it [01:45, 194369.22it/s]







21758333it [01:4

26271879it [02:09, 197356.56it/s]







26291627it [02:09, 194311.92it/s]







26311714it [02:09, 196232.21it/s]







26331833it [02:09, 197693.06it/s]







26351618it [02:09, 195209.07it/s]







26371509it [02:09, 196303.06it/s]







26391692it [02:09, 197925.68it/s]







26411498it [02:09, 188649.82it/s]







26431202it [02:09, 191090.40it/s]







26451039it [02:10, 193216.80it/s]







26470421it [02:10, 191112.24it/s]







26490644it [02:10, 194315.78it/s]







26510902it [02:10, 196723.15it/s]







26530614it [02:10, 194633.40it/s]







26550825it [02:10, 196816.74it/s]







26571043it [02:10, 198395.10it/s]







26590906it [02:10, 195934.85it/s]







26611020it [02:10, 197466.14it/s]







26630786it [02:10, 196364.71it/s]







26650438it [02:11, 194788.41it/s]







26669931it [02:11, 189611.10it/s]







26689263it [02:11, 190708.66it/s]







26708393it [02:11, 190883.96it/s]







26728041it [02:11, 192527.85it/s]







26747522it [02:1

31256758it [02:34, 194367.00it/s]







31276213it [02:34, 192128.63it/s]







31296181it [02:34, 194331.95it/s]







31316265it [02:35, 196237.64it/s]







31335907it [02:35, 192962.24it/s]







31355651it [02:35, 194282.10it/s]







31375269it [02:35, 194846.14it/s]







31394768it [02:35, 192246.74it/s]







31414438it [02:35, 193559.55it/s]







31434547it [02:35, 195757.62it/s]







31454139it [02:35, 193592.16it/s]







31474283it [02:35, 195878.19it/s]







31494514it [02:36, 197762.50it/s]







31514308it [02:36, 195443.32it/s]







31534458it [02:36, 197219.18it/s]







31554650it [02:36, 198605.08it/s]







31574525it [02:36, 191173.52it/s]







31594703it [02:36, 194236.30it/s]







31614644it [02:36, 193869.37it/s]







31634074it [02:36, 193767.18it/s]







31653548it [02:36, 194055.71it/s]







31673176it [02:36, 192845.14it/s]







31692534it [02:37, 193064.24it/s]







31712192it [02:37, 194102.46it/s]







31731666it [02:3

36243685it [03:00, 192779.24it/s]







36263024it [03:00, 191691.88it/s]







36283147it [03:00, 194455.04it/s]







36303219it [03:00, 194144.23it/s]







36322662it [03:00, 194062.29it/s]







36342952it [03:00, 196631.20it/s]







36362638it [03:01, 195842.43it/s]







36382239it [03:01, 195711.41it/s]







36402547it [03:01, 197864.04it/s]







36422348it [03:01, 196083.73it/s]







36442538it [03:01, 197791.51it/s]







36462871it [03:01, 199416.69it/s]







36482825it [03:01, 196429.15it/s]







36502487it [03:01, 188921.47it/s]







36522131it [03:01, 191115.29it/s]







36541301it [03:01, 191046.99it/s]







36561096it [03:02, 193063.89it/s]







36581314it [03:02, 195709.65it/s]







36600918it [03:02, 194160.54it/s]







36620549it [03:02, 194796.46it/s]







36640204it [03:02, 195317.20it/s]







36659749it [03:02, 192392.72it/s]







36679856it [03:02, 194915.75it/s]







36700059it [03:02, 196995.53it/s]







36719779it [03:0

41234615it [03:26, 194334.39it/s]







41254284it [03:26, 195033.25it/s]







41273986it [03:26, 195623.72it/s]







41293555it [03:26, 194062.90it/s]







41313708it [03:26, 196243.24it/s]







41333947it [03:26, 198046.65it/s]







41353764it [03:26, 194361.44it/s]







41373444it [03:26, 195083.93it/s]







41393165it [03:26, 195715.45it/s]







41412749it [03:27, 193852.47it/s]







41432559it [03:27, 195106.30it/s]







41452452it [03:27, 196237.11it/s]







41472085it [03:27, 194662.28it/s]







41491561it [03:27, 193993.40it/s]







41511214it [03:27, 194747.25it/s]







41530695it [03:27, 191959.39it/s]







41550259it [03:27, 193046.55it/s]







41569842it [03:27, 193871.37it/s]







41589238it [03:27, 192098.10it/s]







41609356it [03:28, 194733.83it/s]







41629550it [03:28, 196839.09it/s]







41649251it [03:28, 194580.93it/s]







41668727it [03:28, 194632.45it/s]







41688203it [03:28, 186927.53it/s]







41706970it [03:2

67

In [76]:
np.mean(all_num_pos)

9.244648799287496

In [77]:
all_num_pos = []

with open('sbatch_scripts_generate/dumped/prim_FWD_no_dups/239019/data.prefix.counts_clear', 'r') as expressions:                         # 'data/prim_fwd.train_clean'
    for i, line in tqdm(enumerate(expressions)):
        qa = line.split('|')[1].split('\t')
        if len(qa) == 2:
            q, a = qa
        else:
            print(i,'is broken')
            continue
            
        q = q.split()
        num_ops = 0
        for el in q:
            if el in OPERATORS:
                num_ops += 1
        all_num_pos.append(num_ops)

max(all_num_pos)









0it [00:00, ?it/s]







16507it [00:00, 165063.47it/s]







34900it [00:00, 170302.59it/s]







67395it [00:00, 177192.34it/s]


30

In [78]:
np.mean(all_num_pos)

10.182090659544476